In [1]:
%%capture
!pip install sentence_transformers

In [2]:
%%capture
!pip install datasets

In [3]:
from sentence_transformers import SentenceTransformer, models
from datasets import load_dataset

In [4]:
## Step 1: use an existing language model
word_embedding_model = models.Transformer('distilroberta-base')

## Step 2: use a pool function over the token embeddings
pooling_model = models.Pooling(word_embedding_model.get_word_embedding_dimension())

## Join steps 1 and 2 using the modules argument
model = SentenceTransformer(modules=[word_embedding_model, pooling_model])


# model = SentenceTransformer('sentence-transformers/all-MiniLM-L6-v2')


config.json:   0%|          | 0.00/612 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/90.9M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

In [5]:
from datasets import load_dataset

dataset_id = 'snli'
dataset = load_dataset(dataset_id)

In [6]:
print(f"- The {dataset_id} dataset has {dataset['train'].num_rows} examples.")
print(f"- Each example is a {type(dataset['train'][0])} with a {type(dataset['train'][0]['premise'])} as value.")
print(f"- Examples look like this: {dataset['train'][0]}")

- The snli dataset has 550152 examples.
- Each example is a <class 'dict'> with a <class 'str'> as value.
- Examples look like this: {'premise': 'A person on a horse jumps over a broken down airplane.', 'hypothesis': 'A person is training his horse for a competition.', 'label': 1}


In [7]:
from sentence_transformers import InputExample

train_examples = []
train_data = dataset['train']
# For agility we only 1/2 of our available data
n_examples = dataset['train'].num_rows // 2

for i in range(n_examples):
  example = train_data[i]
  train_examples.append(InputExample(texts=[example['premise'], example['hypothesis']], label=float(example['label'])))

In [8]:
print(f"We have a {type(train_examples)} of length {len(train_examples)} containing {type(train_examples[0])}'s.")

We have a <class 'list'> of length 275076 containing <class 'sentence_transformers.readers.InputExample.InputExample'>'s.


In [9]:
from torch.utils.data import DataLoader

train_dataloader = DataLoader(train_examples, shuffle=True, batch_size=16)

In [10]:
from sentence_transformers import losses

train_loss = losses.CosineSimilarityLoss(model=model)

In [14]:
num_epochs = 1

warmup_steps = int(len(train_dataloader) * num_epochs * 0.1) #10% of train data

In [ ]:
model.fit(train_objectives=[(train_dataloader, train_loss)],
          epochs=num_epochs,
          warmup_steps=warmup_steps)

Epoch:   0%|          | 0/1 [00:00<?, ?it/s]

Iteration:   0%|          | 0/17193 [00:00<?, ?it/s]